James Harrison, 2022-04-20

This notebook is used to produce the following aggregates:
- Resident count per admin3 last month
- Home relocation counts from month-before-last to last month, per pair of admin3s
- Count of total subscribers active anywhere in the country at any time during the month

'Last month' is the most recently-ended full calendar month before today.

These aggregates can be used to produce mobility indicators related to residence and home relocation.

In [ ]:
import datetime
import warnings
from pathlib import Path

import flowclient as fc
import flowmachine as fm
import pandas as pd
from flowclient import flows, spatial_aggregate, unique_subscriber_counts
from get_secret_or_env_var import environ
from utils import (
    check_data_availability_for_home_locations,
    find_dates_to_exclude_monthly,
    get_date_in_month,
    monthly_home_location_spec,
    monthly_subscriber_subset_query,
    run_query_and_write_result,
)

# Setup

## Parameters

In [ ]:
datetime_now = datetime.datetime.now()
datetime_now

In [ ]:
# Parameters
author = "James Harrison <james.harrison@flowminder.org>"

start_date = (
    None  # First day of the month (defaults to start of first full month before now)
)
month_start_day = None  # If start_date is not specified, start on this day of the month (defaults to 1, i.e. first day of a calendar month)
window_length = 7  # Length in days of the rolling window used to compute modal locations and average call days
min_call_days = (
    3  # Minimal number of average days in a window a subscriber was sighted on
)
min_percent_of_data_dates = 60  # Minimum percentage of days of data that must be present for a month to be included
max_data_gap = (
    6  # Maximum length (in days) of allowed data gap for a month to be included
)
max_empty_windows = (
    None  # Maximum number of entirely-empty windows allowed for a month to be included
)
latest_truncation_threshold = (
    "18:00:00"  # Threshold for excluding temporally-truncated data
)

aggregation_unit = "admin3"  # Spatial aggregation unit
mapping_table = "geography.cell_to_admin_via_clusters_1km_20221025"
geom_table = None
geom_table_join_column = None
event_types = ["calls"]  # Event types to use
flowmachine_log_level = "info"  # Flowmachine log level
shared_data_dir = "./"  # Writable output directory
outputs_subdir = "aggregates/residence_relocations"  # Subdirectory of shared data dir to which results of aggregate queries will be written
output_format = "csv"  # 'csv' or 'netcdf'
overwrite = False  # Set True to overwrite previously-saved aggregates for this month (with overwrite=False, conflicting aggregate files will be renamed)
calculate_relocations = True  # Set False to skip running the home relocations aggregate
calculate_total_subscribers = (
    True  # Set False to skip running the total subscribers aggregate
)
require_latest_data = True  # If True, computation will not proceed if the last required day of data is later than the most recent available date

In [ ]:
# If start date is not specified, calculate start date from time now
if start_date is not None:
    if month_start_day is not None:
        raise ValueError(
            "Only one of 'start_date' or 'month_start_day' may be specified"
        )
else:
    if month_start_day is None:
        month_start_day = 1
    start_date = get_date_in_month(
        datetime_now,
        day_of_month=month_start_day,
        month_offset=(
            -1 if datetime_now.day >= month_start_day else -2
        ),  # If day of month now is before month_start_day, need to offset by 2 months to get a complete month (this ignores ingestion time)
    )

In [ ]:
# Construct outputs path (we don't actually create the dir until we're ready to start writing outputs later)
outputs_path = (
    Path(shared_data_dir)
    / outputs_subdir
    / f"residence_relocations_aggregates_{start_date}"
)

outputs_path

In [ ]:
# TODO: We're now specifying start_date directly, so rewrite the rest of the notebook to use this instead of date_today and month_start_day
date_today = pd.Timestamp(start_date) + pd.DateOffset(months=1)
month_start_day = pd.Timestamp(start_date).day

## Connect

In [ ]:
fc_conn = fc.connect(
    url=environ["FLOWAPI_URL"],
    ssl_certificate=False,  # Workaround pending https://github.com/Flowminder/flowpyter-task/issues/35
    token=environ["FLOWAPI_TOKEN"],
)

In [ ]:
fm.connect(
    flowdb_connection_pool_overflow=20,
    flowdb_connection_pool_size=5,
    log_level=flowmachine_log_level,
)

## Check dates

In [ ]:
# Find earliest date for which all required event types are available
# (so that we don't try to look back indefinitely for a complete 'from' month)
available_dates = fc.get_available_dates(connection=fc_conn)
available_dates_all_eventtypes = set(available_dates[event_types[0]]).intersection(
    *(set(available_dates[event_type]) for event_type in event_types[1:])
)
earliest_available_date = min(available_dates_all_eventtypes)

In [ ]:
available_aggregates = set()

# Check data completeness for resident counts
to_month_offset = -1
(
    to_month_available,
    to_lookback_month_available,
    to_lookback_n_months,
) = check_data_availability_for_home_locations(
    date_today,
    flowdb_connection=fm.core.context.get_db(),
    month_offset=to_month_offset,
    month_start_day=month_start_day,
    window_length=window_length,
    event_types=event_types,
    latest_truncation_threshold=latest_truncation_threshold,
    fail_on_missing_latest=require_latest_data,
    min_percent_of_dates=min_percent_of_data_dates,
    max_allowed_gap=max_data_gap,
    max_empty_windows=max_empty_windows,
    min_median_included_days_per_window=min_call_days,
)
if to_month_available:
    available_aggregates.add("resident-counts")
    if calculate_total_subscribers:
        available_aggregates.add("total-subscribers")
    required_subsets = [to_month_offset]
    if to_lookback_month_available:
        to_lookback_month_offset = to_month_offset - to_lookback_n_months
        required_subsets.append(to_lookback_month_offset)
    else:
        warnings.warn(
            f"Home location for resident-counts will be calculated from just one month's majority location (no lookback)"
        )
else:
    warnings.warn(f"Insufficient complete months to run 'resident-counts' aggregate")

# Check data completeness for home relocations
if calculate_relocations and to_month_available:
    if to_lookback_month_available:
        from_month_offset = to_lookback_month_offset
    else:
        from_month_offset = -4
    (
        from_month_available,
        from_lookback_month_available,
        from_lookback_n_months,
    ) = check_data_availability_for_home_locations(
        date_today,
        flowdb_connection=fm.core.context.get_db(),
        month_offset=from_month_offset,
        month_start_day=month_start_day,
        window_length=window_length,
        event_types=event_types,
        latest_truncation_threshold=latest_truncation_threshold,
        fail_on_missing_latest=require_latest_data,
        min_percent_of_dates=min_percent_of_data_dates,
        max_allowed_gap=max_data_gap,
        max_empty_windows=max_empty_windows,
        min_median_included_days_per_window=min_call_days,
    )
    while (not from_month_available) and (
        get_date_in_month(
            date_today, day_of_month=month_start_day, month_offset=from_month_offset
        )
        > pd.Timestamp(earliest_available_date).date()
    ):
        # Find the most recent previous month for which sufficient data are available
        from_month_offset -= 1
        (
            from_month_available,
            from_lookback_month_available,
            from_lookback_n_months,
        ) = check_data_availability_for_home_locations(
            date_today,
            flowdb_connection=fm.core.context.get_db(),
            month_offset=from_month_offset,
            month_start_day=month_start_day,
            window_length=window_length,
            event_types=event_types,
            latest_truncation_threshold=latest_truncation_threshold,
            fail_on_missing_latest=require_latest_data,
            min_percent_of_dates=min_percent_of_data_dates,
            max_allowed_gap=max_data_gap,
            max_empty_windows=max_empty_windows,
            min_median_included_days_per_window=min_call_days,
        )
    if from_month_available:
        available_aggregates.add("home-relocations")
        if not to_lookback_month_available:
            warnings.warn(
                f"Home location for 'to' month of home-relocations will be calculated from just one month's majority location (no lookback)"
            )
            required_subsets.append(from_month_offset)
        if from_lookback_month_available:
            from_lookback_month_offset = from_month_offset - from_lookback_n_months
            required_subsets.append(from_lookback_month_offset)
        else:
            warnings.warn(
                f"Home location for 'from' month of home-relocations will be calculated from just one month's majority location (no lookback)"
            )
    else:
        warnings.warn(
            f"Insufficient complete months to run 'home-relocations' aggregate"
        )
elif calculate_relocations:
    warnings.warn(f"Insufficient complete months to run 'home-relocations' aggregate")

if not available_aggregates:
    raise ValueError("Insufficient complete months to run any aggregates")

required_subsets

In [ ]:
sorted_month_offsets = sorted(required_subsets, reverse=True)
dates_to_skip = find_dates_to_exclude_monthly(
    date_today,
    flowdb_connection=fm.core.context.get_db(),
    month_offset=sorted_month_offsets[0],
    month_start_day=month_start_day,
    window_length=window_length,
    event_types=event_types,
    latest_truncation_threshold=latest_truncation_threshold,
    fail_on_missing_latest=require_latest_data,
).union(
    *(
        find_dates_to_exclude_monthly(
            date_today,
            flowdb_connection=fm.core.context.get_db(),
            month_offset=mo,
            month_start_day=month_start_day,
            window_length=window_length,
            event_types=event_types,
            latest_truncation_threshold=latest_truncation_threshold,
            fail_on_missing_latest=require_latest_data,
        )
        for mo in sorted_month_offsets[1:]
    )
)
dates_to_skip

# Subscriber subsets

Subscriber subsets have to be defined and run using flowmachine directly, and then the query IDs can be used to subset FlowAPI queries.

## Define subscriber subset queries

In [ ]:
tables = [f"events.{event_type}" for event_type in event_types]

In [ ]:
# Convert FlowAPI aggregation unit parameters to a flowmachine spatial unit
if "admin" not in aggregation_unit:
    raise NotImplementedError(
        "This notebook cannot yet handle non-admin aggregation units for the subscriber subset definition"
    )

spatial_unit = fm.core.spatial_unit.make_spatial_unit(
    spatial_unit_type="admin",
    level=int(aggregation_unit[-1]),
    mapping_table=mapping_table,
    geom_table=geom_table,
    geom_table_join_on=geom_table_join_column,
)

In [ ]:
subscriber_subsets_and_intermediates = [
    monthly_subscriber_subset_query(
        date_today,
        month_offset=month_offset,
        month_start_day=month_start_day,
        window_length=window_length,
        min_call_days=min_call_days,
        spatial_unit=spatial_unit,
        tables=tables,
        dates_to_exclude=dates_to_skip,
    )
    for month_offset in required_subsets
]

## Run subscriber subset queries

In [ ]:
subset_futs = [
    subset.store(store_dependencies=True)
    for subset, intermediates in subscriber_subsets_and_intermediates
    if not subset.is_stored
]

Wait until all subsets have been calculated.

In [ ]:
import concurrent.futures

concurrent.futures.wait(subset_futs)

In [ ]:
[len(subset) for subset, intermediates in subscriber_subsets_and_intermediates]

## Wrap in Table objects so that flowmachine server can unpickle

In [ ]:
subscriber_subset_tables = [
    subset.get_table() for subset, intermediates in subscriber_subsets_and_intermediates
]

In [ ]:
subscriber_subset_query_ids = [subset.query_id for subset in subscriber_subset_tables]

In [ ]:
subscriber_subset_query_ids

Subset query ids can now be passed on to API queries.

# FlowAPI side

## Define queries

In [ ]:
api_queries = {}

### Home location sub-queries

In [ ]:
if ("resident-counts" in available_aggregates) or (
    "home-relocations" in available_aggregates
):
    to_month_home_location_spec = monthly_home_location_spec(
        date_today,
        month_offset=to_month_offset,
        month_start_day=month_start_day,
        window_length=window_length,
        lookback_n_months=to_lookback_n_months,
        aggregation_unit=aggregation_unit,
        mapping_table=mapping_table,
        geom_table=geom_table,
        geom_table_join_column=geom_table_join_column,
        this_month_subscriber_subset=subscriber_subset_query_ids[0],
        last_month_subscriber_subset=(
            subscriber_subset_query_ids[1] if to_lookback_month_available else None
        ),
        event_types=event_types,
        dates_to_exclude=dates_to_skip,
    )

if "home-relocations" in available_aggregates:
    from_month_home_location_spec = monthly_home_location_spec(
        date_today,
        month_offset=from_month_offset,
        month_start_day=month_start_day,
        window_length=window_length,
        lookback_n_months=from_lookback_n_months,
        aggregation_unit=aggregation_unit,
        mapping_table=mapping_table,
        geom_table=geom_table,
        geom_table_join_column=geom_table_join_column,
        this_month_subscriber_subset=subscriber_subset_query_ids[1],
        last_month_subscriber_subset=(
            subscriber_subset_query_ids[2] if from_lookback_month_available else None
        ),
        event_types=event_types,
        dates_to_exclude=dates_to_skip,
    )

### Resident counts

In [ ]:
if "resident-counts" in available_aggregates:
    api_queries["resident-counts"] = spatial_aggregate(
        connection=fc_conn,
        locations=to_month_home_location_spec,
    )

### Home relocations matrix

In [ ]:
if "home-relocations" in available_aggregates:
    # Run home relocations query twice, once with an inner join and once with an outer join,
    # so that we get counts of both (unlocatable -> A) and (unlocatable+inactive -> A)
    for join_type in ["inner", "full outer"]:
        api_queries[f"home-relocations_{join_type.replace(' ', '-')}"] = flows(
            connection=fc_conn,
            from_location=from_month_home_location_spec,
            to_location=to_month_home_location_spec,
            join_type=join_type,
        )

### Total subscribers

In [ ]:
if "total-subscribers" in available_aggregates:
    # Note: not using a subscriber subset here - we want to count all subscribers who were active even once
    api_queries["total-subscribers"] = unique_subscriber_counts(
        connection=fc_conn,
        start_date=str(
            get_date_in_month(
                date_today, day_of_month=month_start_day, month_offset=to_month_offset
            )
        ),
        end_date=str(
            get_date_in_month(
                date_today,
                day_of_month=month_start_day,
                month_offset=to_month_offset + 1,
            )
        ),
        aggregation_unit="admin0",
        mapping_table=mapping_table,
        event_types=event_types,
    )

## Run queries

In [ ]:
for label, query in api_queries.items():
    print(f"Setting '{label}' query running...")
    query.run()
print("All queries are running")

## Get results and write to files

In [ ]:
to_month_date_string = str(
    get_date_in_month(
        date_today, day_of_month=month_start_day, month_offset=to_month_offset
    )
)
if calculate_relocations:
    from_month_date_string = str(
        get_date_in_month(
            date_today, day_of_month=month_start_day, month_offset=from_month_offset
        )
    )

In [ ]:
common_additional_attrs = {
    "author": author,
    "redacted": True,
    "excluded_dates": sorted(dates_to_skip),
}

In [ ]:
outputs_path.mkdir(exist_ok=True, parents=True)

In [ ]:
for label, query in api_queries.items():
    print(f"Getting result of '{label}' query...")
    if "relocations" in label:
        date_attrs = dict(
            month_start_date_from=from_month_date_string,
            month_start_date_to=to_month_date_string,
        )
        date_tag = f"from{from_month_date_string}_to{to_month_date_string}"
    else:
        date_attrs = dict(month_start_date=to_month_date_string)
        date_tag = to_month_date_string
    run_query_and_write_result(
        query,
        filepath=outputs_path / f"{label}_{date_tag}",
        overwrite=overwrite,
        file_format=output_format,
        additional_attrs={**common_additional_attrs, **date_attrs},
    )
print("All queries completed")